# Compare with Actual Distilbert

In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from distilbert_by_hand import DistilBertByHand

In [4]:
# their_transformer.eval()  # evaluation mode
distilbert_weights = their_transformer.state_dict()
# for key in distilbert_weights.keys():
#     if 'layer.0' in key:
#         print(key, distilbert_weights[key].shape)

In [ ]:
sentence = "The cat sat on the mat."

# My implementation
my_transformer = DistilBertByHand()

# Actual implementation
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
their_transformer = DistilBertModel.from_pretrained('distilbert-base-uncased')
inputs = tokenizer(sentence, return_tensors="pt")

# Compare the outputs
my_output = my_transformer(sentence)
with torch.no_grad():  
    their_output = their_transformer(**inputs)

print(my_output)
print(their_output[0][0])

In [ ]:
their_transformer.eval()  # evaluation mode

In [5]:
# Compare the outputs


# My ouputs
my_output = my_transformer(sentence)
my_hidden_states = my_transformer.hidden_states
my_attention_weights = my_transformer.attention_weights
print(my_output)

# Their outputs
with torch.no_grad():  
    their_output = their_transformer(**inputs)
    their_attention_weights = their_output.attentions
print(their_output[0][0])

head_divided_weight_matrix shape:  torch.Size([12, 64, 768])
head_divided_weight_matrix shape:  torch.Size([12, 64, 768])
head_divided_weight_matrix shape:  torch.Size([12, 64, 768])
Q shape:  torch.Size([12, 9, 64])
K shape:  torch.Size([12, 9, 64])
V shape:  torch.Size([12, 9, 64])
A shape:  torch.Size([12, 9, 9])
V shape:  torch.Size([12, 9, 64])
V shape:  torch.Size([9, 768])
tensor([[ 0.4260,  0.8195,  1.9080,  ...,  0.2898, -0.9066, -0.3869],
        [ 0.2259, -0.0940,  0.1709,  ..., -0.9786, -0.3274, -0.0714],
        [ 0.1300, -0.2636,  0.1164,  ...,  0.2267,  0.3086,  0.2580],
        ...,
        [-0.6407,  0.3197,  0.5364,  ..., -0.5793,  0.0201,  0.2133],
        [ 0.1996, -0.1013,  1.0933,  ..., -0.0736,  0.0613, -0.3893],
        [ 0.6916,  0.4632,  1.5045,  ...,  0.1637, -0.3426,  0.0680]])
tensor([[-0.2713, -0.0781, -0.0216,  ..., -0.0853,  0.4197,  0.1664],
        [-0.2253,  0.0514, -0.1776,  ..., -0.0192,  1.0410, -0.4474],
        [-0.1074, -0.0692,  0.1663,  ..., -

In [6]:
print(my_attention_weights[0].shape)
print(their_attention_weights[0][0].shape)

their_A = their_attention_weights[0][0]
my_A = my_attention_weights[0]
rmse(their_A, my_A)

torch.Size([12, 9, 9])
torch.Size([12, 9, 9])
RMSE:  tensor(0.0002)


# Investigate where I diverge

In [7]:
# Investigate where we diverge

# For space, I stop at the first layer because I can already see the difference, which lies somewhere between
# the attention weights and the output of the first layer
print('-------------------------------------------------------------')
for i in range(2): #7):
    if i == 0:
        print('Embedding Layer')
        rmse(my_hidden_states[i],their_output['hidden_states'][i][0])
        print('-------------------------------------------------------------')
    else:
        print('Layer ', i-1)
        print('Attention')
        rmse(my_attention_weights[i-1],their_attention_weights[i-1][0])
        print('Hidden Layer Output')
        rmse(my_hidden_states[i],their_output['hidden_states'][i][0])
        print('-------------------------------------------------------------')

-------------------------------------------------------------
Embedding Layer
RMSE:  tensor(0.0004)
-------------------------------------------------------------
Layer  0
Attention
RMSE:  tensor(0.0002)
Hidden Layer Output
RMSE:  tensor(0.5929)
-------------------------------------------------------------


## So it's something in between attention and the end of the block.

# Hook Method

## I've narrowed it down to the residual (output of self-attention before adding to X)

In [8]:
their_A = their_attention_weights[0][0]
my_A = my_attention_weights[0]
rmse(their_A, my_A)
# their_V  = 

# my_V = my_transformer.mine_for_comparison

RMSE:  tensor(0.0002)


# Investigate AV

In [9]:
# def hook(module, input, output):
#     global hook_output
#     hook_output = output


# model = DistilBertModel.from_pretrained('distilbert-base-uncased')
# hook_handle = model.transformer.layer[0].attention.v_lin.register_forward_hook(hook)

# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# inputs = tokenizer(sentence, return_tensors="pt")

# with torch.no_grad():
#     output = model(**inputs)
    
# my_V_weights = my_transformer.mine_for_comparison
# my_V_weights = my_V_weights.view(1,9,768)
# their_V_weights = hook_output

# hook_handle.remove()

In [10]:
# print(my_V_weights.shape)
# print(their_V_weights.shape)

In [11]:
distilbert_weights.keys()

odict_keys(['embeddings.word_embeddings.weight', 'embeddings.position_embeddings.weight', 'embeddings.LayerNorm.weight', 'embeddings.LayerNorm.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.1.attention.q_lin.weight', 'transformer.layer.1.attention.q_lin.bias', 'transformer.layer.1.attention.k_lin.weight', 'transformer.layer

In [12]:
print(distilbert_weights['transformer.layer.0.attention.q_lin.bias'].shape)
print(distilbert_weights['transformer.layer.0.attention.k_lin.weight'].shape)
print(distilbert_weights['transformer.layer.0.attention.v_lin.weight'].shape)

torch.Size([768])
torch.Size([768, 768])
torch.Size([768, 768])


In [13]:
# my_V_weights = my_V_weights.squeeze(1)
# print(my_V_weights.shape)

In [14]:
# rmse(my_V_weights,their_V_weights)

In [15]:
# their_transformer.transformer.layer[0]

In [16]:
# hook_handle = model.transformer.layer[0].attention.v_lin.register_forward_hook(hook)

In [17]:
# W_out_lin = distilbert_weights['transformer.layer.0.attention.out_lin.weight']
# b_out_lin = distilbert_weights['transformer.layer.0.attention.out_lin.bias']
# b_out_lin_matrix = b_out_lin.repeat(len(inputs), 1)

In [18]:
# W_out_lin + b_out_lin